In [1]:
import math
import numpy as np
import pandas as pd
import os
import glob

from pykrx import stock
from pykrx import bond

import time
from time import sleep
from datetime import datetime
from datetime import timedelta

from pyarrow import csv
import pyarrow as pa
import pyarrow.parquet as pq

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import psycopg2 as pg2
from sqlalchemy import create_engine

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage

import google.generativeai as genai

import warnings
warnings.filterwarnings('ignore')

# 경로 변경
os.chdir('/home/shjj08choi4/finance_mlops')


# 서비스 계정 키 JSON 파일 경로
key_path = glob.glob("key_value/*.json")[0]

# Credentials 객체 생성
credentials = service_account.Credentials.from_service_account_file(key_path)

# 빅쿼리 정보
project_id = 'owenchoi-404302'
dataset_id = 'finance_mlops'


# BigQuery 클라이언트 객체 생성
client = bigquery.Client(credentials = credentials, 
                         project = credentials.project_id)


# GCP 클라이언트 객체 생성
storage_client = storage.Client(credentials = credentials,
                         project = credentials.project_id)
bucket_name = 'finance-mlops-proj'    # 서비스 계정 생성한 bucket 이름 입력

# Postgresql 연결
db_connect_info = pd.read_csv('key_value/db_connect_info.csv')
username = db_connect_info['username'][0]
password = db_connect_info['password'][0]
host = db_connect_info['host'][0]
database = db_connect_info['database'][0]
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:5432/{database}')


now = datetime.now()
# now = now + timedelta(days=-2)
today_date1 = now.strftime('%Y%m%d')
today_date2 = now.strftime('%Y-%m-%d')
today_date_time_csv = now.strftime("%Y%m%d_%H%M")

In [2]:
gemini_result_kospi_20240115 = pd.read_csv('data_crawler/dashboard/gemini_result_kosdaq_20240115.csv', 
                                          dtype = {'ticker':str})

In [5]:
# Google Storage 적재
source_file_name = f'data_crawler/dashboard/gemini_result_kosdaq_20240115.csv'    # GCP에 업로드할 파일 절대경로
destination_blob_name = f'data_crawler/dashboard/gemini_result_kosdaq_20240115.csv'    # 업로드할 파일을 GCP에 저장할 때의 이름
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename(source_file_name)

In [12]:
gemini_result_kospi_20240115[gemini_result_kospi_20240115['ticker'] == '373220'].reset_index(drop = True)['response_msg'][0]

'**증권 보고서**\n\n**날짜:** 2024년 1월 15일\n\n**종목 정보:** LG 에너지 솔루션\n\n**등락률:**\n\nLG 에너지 솔루션의 주가는 2024년 1월 5일 전 거래일 대비 2.5% 하락했습니다. 이는 동 기간 동안 코스피가 0.5% 상승한 것과 대조적입니다.\n\n**매수/매도 신호:**\n\n* LG 에너지 솔루션은 역배열(매도)과 MACD 하향 돌파(매도) 신호를 생성했습니다.\n\n**결론:**\n\nLG 에너지 솔루션의 주가는 하락세로 전환했으며 매수/매도 신호는 매도를 권장합니다. 따라서 투자자들은 주의가 필요합니다.'